In [1]:
import os
import locale
import math

import pandas as pd
import numpy as np

from dotenv import load_dotenv
from utils_ccd import get_connection
from docxtpl import DocxTemplate

load_dotenv()
conn = get_connection()

locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')

from antecedentes import create_antecedentes_doc

import pypdf
from pathlib import Path

from langchain_openai import AzureOpenAI, AzureChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate, FewShotChatMessagePromptTemplate

from docx2pdf import convert

/home/eduardo/Dev/ccd/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import subprocess

def generate_pdf(doc_path, path):

    subprocess.call(['soffice',
                  '--headless',
                 '--convert-to',
                 'pdf',
                 '--outdir',
                 path,
                 doc_path])
    return doc_path

In [3]:
def get_file_path(row):
    return Path('/media/informacoes_pdf/') / row['setor'].strip() / row['arquivo']


In [4]:
def get_pdf_text(row):
    arquivo = get_file_path(row)
    print(f'File {arquivo} to pypdf text')
    try:
        pdf = pypdf.PdfReader(arquivo)
        text = []
        for page in pdf.pages:
            text.append(page.extract_text())
        return ' '.join(text)
    except FileNotFoundError:
        print(f'File not found: {arquivo}')
        return ''

In [ ]:
antecedentes_sql = ''' 

SELECT DISTINCT pro.numero_processo,
       pro.ano_processo,
       pro.assunto,
       pro.interessado,
       pm.Descricao as marcador      
       
FROM processo.dbo.Processos pro
INNER JOIN processo.dbo.Pro_MarcadorProcesso pmp ON pmp.IdProcesso = pro.IdProcesso
INNER JOIN processo.dbo.Pro_Marcador pm ON pmp.IdMarcador = pm.IdMarcador
LEFT JOIN processo.dbo.Exe_Debito ed ON ed.IdProcessoExecucao = pro.IdProcesso
LEFT JOIN processo.dbo.Exe_DebitoPessoa edp ON edp.IDDebito = ed.IdDebito
LEFT JOIN processo.dbo.GenPessoa gp ON gp.IdPessoa = edp.IDPessoa
WHERE pro.setor_atual = 'CCD'
AND lower(pm.Descricao) like '%antecedente%'
'''
processos_antecedentes = pd.read_sql(antecedentes_sql, conn)

/tmp/ipykernel_131912/146702693.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  processos_antecedentes = pd.read_sql(antecedentes_sql, conn)


In [ ]:
processos_antecedentes

,numero_processo,ano_processo,assunto,interessado,marcador
0,000761,2022,APURAÇÃO DE RESPONSABILIDADE EM ATENDIMENTO AO...,PREFEITURA MUNICIPAL DE CANGUARETAMA ...,Antecedentes
1,000913,2023,COMUNICAÇÃO DE IRREGULARIDADE ...,SIGILOSO ...,Antecedentes
2,001749,2019,"CONTAS DO CHEFE DO PODER EXECUTIVO MUNICIPAL, ...",PREFEITURA MUNICIPAL DE VIÇOSA ...,Antecedentes
3,003277,2020,AUMENTO REMUNERATÓRIO DURANTE A PANDEMIA - REM...,CÂMARA MUNICIPAL DE ANTÔNIO MARTINS ...,Antecedentes
4,003980,2022,"APURAÇÃO DE RESPONSABILIDADE, REFERENTE AO PRO...",PREFEITURA MUNICIPAL DE CEARÁ-MIRIM ...,Antecedentes
5,004072,2021,MONITORAMENTO ...,AGÊNCIA DE FOMENTO DO RN S/A ...,Antecedentes
6,004728,2022,DENÚNCIA ...,FRANCINALDO MOREIRA DA SILVA ...,Antecedentes
7,005119,2021,APURAÇÃO DE RESPONSABILIDADE REFERENTE A ATRAS...,SECRETARIA MUNICIPAL DE MEIO AMBIENTE E URBANI...,Antecedentes
8,005181,2020,AUDITORIA DE CONFORMIDADE-EXERCÍCIO 2018 ...,PREFEITURA MUNICIPAL DE MOSSORÓ ...,Antecedentes
9,006621,2015,CONTAS DO CHEFE DO PODER EXECUTIVO REFERENTE A...,PREF.MUN.UPANEMA ...,Antecedentes


In [ ]:
sql_info_antecedentes_gaana = ''' 
WITH RankedData AS (
    SELECT 
    	ai.setor,
        concat(pro.numero_processo, '/', pro.ano_processo) as processo,
        pro.assunto,
        pro.interessado,
        concat(rtrim(ai.setor),'_',
        ai.numero_processo ,'_',
        ai.ano_processo,'_',RIGHT(concat('0000',ai.ordem),4),'.pdf') as arquivo,
        ai.datapublicacao, 
        ppe.SequencialProcessoEvento, 
        ai.numero_processo, 
        ai.ano_processo,
        ROW_NUMBER() OVER (
            PARTITION BY CONCAT(ai.numero_processo, '/', ai.ano_processo)
            ORDER BY ppe.SequencialProcessoEvento DESC
        ) AS rn
    FROM processo.dbo.vw_Ata_Informacao ai 
    LEFT JOIN processo.dbo.Pro_ProcessoEvento ppe 
        ON ai.idInformacao = ppe.IdInformacao
    LEFT JOIN processo.dbo.Processos pro
    	ON pro.numero_processo = ai.numero_processo AND pro.ano_processo = ai.ano_processo
    WHERE CONCAT(ai.numero_processo, '/', ai.ano_processo) IN (
        SELECT CONCAT(pro.numero_processo, '/', pro.ano_processo)
        FROM processo.dbo.Processos pro
        INNER JOIN processo.dbo.Pro_MarcadorProcesso pmp 
            ON pmp.IdProcesso = pro.IdProcesso
        INNER JOIN processo.dbo.Pro_Marcador pm 
            ON pmp.IdMarcador = pm.IdMarcador
        LEFT JOIN processo.dbo.Exe_Debito ed 
            ON ed.IdProcessoExecucao = pro.IdProcesso
        LEFT JOIN processo.dbo.Exe_DebitoPessoa edp 
            ON edp.IDDebito = ed.IdDebito
        LEFT JOIN processo.dbo.GenPessoa gp 
            ON gp.IdPessoa = edp.IDPessoa
        WHERE pro.setor_atual = 'CCD'
        AND lower(pm.Descricao) LIKE '%antecedente%'
    )
    AND ai.setor LIKE '%GAANA%'
)
SELECT * 
FROM RankedData 
WHERE rn = 1
ORDER BY numero_processo, ano_processo;


'''
df_info_gaana = pd.read_sql(sql_info_antecedentes_gaana, conn)

/tmp/ipykernel_131912/3946558724.py:49: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_info_gaana = pd.read_sql(sql_info_antecedentes_gaana, conn)


In [ ]:
def get_transito(nome):
    with open("consultas/processos_transito_cpf.sql") as f:
        query = f.read()
    with get_connection() as conn:
        q = query.format(nome=nome)
        #print(q)
        df = pd.read_sql(q, conn)
        df['valor_original'] = df['valor_original'].apply(lambda x: locale.currency(x, grouping=True, symbol=False) if x and not math.isnan(x) else '-')
        df['valor_atualizado'] = df['valor_atualizado'].apply(lambda x: locale.currency(x, grouping=True, symbol=False) if x and not math.isnan(x) else '-')
        df.fillna('', inplace=True)
        return df.to_dict(orient='records')
    
def get_processos_transito_by_cpf(pessoas_cpfs):
    transitos = []
    for pessoa in pessoas_cpfs:
        if pessoa['nome']:
            transitos.append({'nome': pessoa['nome'], 'cpf': pessoa['cpf'], 'transitos': get_transito(pessoa['nome'])})
    return transitos   

def get_genpessoa(pessoa_strc):
    cpfs = []
    for pessoa in pessoa_strc.pessoas:
        nome_pessoa = pessoa.nome_pessoa
        sql = f'''
        SELECT gp.Documento
            FROM Processo.dbo.GenPessoa gp 
            WHERE lower(gp.nome) = '{nome_pessoa.lower()}'
        '''
        ret = pd.read_sql(sql, get_connection())
        if ret.shape[0] == 0 or ret.shape[0] > 1:
            cpfs.append({'nome': nome_pessoa, 'cpf': None})
        else:
            cpfs.append({'nome': nome_pessoa, 'cpf': ret.iloc[0]['Documento']})
    return cpfs

def is_vazio(valores):
    return all([not bool(v['transitos']) for v in valores])
    

In [ ]:
df_info_gaana['texto_pdf'] = df_info_gaana.apply(get_pdf_text, axis=1)

File /media/informacoes_pdf/GAANA/GAANA_000761_2022_0028.pdf to pypdf text
File /media/informacoes_pdf/GAANA/GAANA_000913_2023_0027.pdf to pypdf text
File /media/informacoes_pdf/GAANA/GAANA_001749_2019_0042.pdf to pypdf text
File /media/informacoes_pdf/GAANA/GAANA_003277_2020_0124.pdf to pypdf text
File /media/informacoes_pdf/GAANA/GAANA_003980_2022_0029.pdf to pypdf text
File /media/informacoes_pdf/GAANA/GAANA_004072_2021_0039.pdf to pypdf text
File /media/informacoes_pdf/GAANA/GAANA_004728_2022_0049.pdf to pypdf text
File /media/informacoes_pdf/GAANA/GAANA_005119_2021_0016.pdf to pypdf text
File /media/informacoes_pdf/GAANA/GAANA_005181_2020_0084.pdf to pypdf text
File /media/informacoes_pdf/GAANA/GAANA_006621_2015_0037.pdf to pypdf text
File /media/informacoes_pdf/GAANA/GAANA_200086_2023_0045.pdf to pypdf text
File /media/informacoes_pdf/GAANA/GAANA_200123_2023_0038.pdf to pypdf text


In [ ]:
llm = AzureChatOpenAI(model_name="gpt-4o")

In [ ]:
from typing import List

from pydantic import BaseModel, Field

class PessoaAntecedentes(BaseModel):
  """Pessoa cujo antecedente deve ser buscado"""
  nome_pessoa: str = Field(description="Nome da pessoa para buscar antecedentes")

class PessoasAntecedentes(BaseModel):
  """Lista de pessoas cujos antecedentes devem ser buscados"""
  pessoas: List[PessoaAntecedentes] = Field(description="Lista de pessoas para buscar antecedentes")


In [ ]:
def get_pessoas_llm(texto_despacho):

  prompt = PromptTemplate.from_template("""
    Você é um agente que identifica listas de pessoas em requisições de antecedentes. 
      Você recebeu uma requisição para buscar antecedentes de pessoas em um processo. Remova qualquer vocativo do nome da pessoa, tal como Sr., Sra., Dr., Dra., etc.
      O texto da requisição é o seguinte:
      "{input}"
      Encontre os antecedentes das pessoas listadas na requisição.

    Sua resposta:
    """)

  structured_llm = llm.with_structured_output(schema=PessoasAntecedentes)
  chain = prompt | structured_llm
  return chain.invoke(texto_despacho)

In [15]:
df_info_gaana['pessoas'] = df_info_gaana['texto_pdf'].apply(get_pessoas_llm)

In [16]:
df_info_gaana['cpf'] = df_info_gaana['pessoas'].apply(get_genpessoa)

/tmp/ipykernel_131912/2334643007.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ret = pd.read_sql(sql, get_connection())
/tmp/ipykernel_131912/2334643007.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ret = pd.read_sql(sql, get_connection())
/tmp/ipykernel_131912/2334643007.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ret = pd.read_sql(sql, get_connection())
/tmp/ipykernel_131912/2334643007.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlit

In [24]:
df_info_gaana['valores'] = df_info_gaana['cpf'].apply(get_processos_transito_by_cpf)

/tmp/ipykernel_131912/2334643007.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(q, conn)
/tmp/ipykernel_131912/2334643007.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(q, conn)
/tmp/ipykernel_131912/2334643007.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(q, conn)
/tmp/ipykernel_131912/2334643007.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are no

In [25]:
df_info_gaana

,setor,processo,assunto,interessado,arquivo,datapublicacao,SequencialProcessoEvento,numero_processo,ano_processo,rn,texto_pdf,pessoas,cpf,valores
0,GAANA,000761/2022,APURAÇÃO DE RESPONSABILIDADE EM ATENDIMENTO AO...,PREFEITURA MUNICIPAL DE CANGUARETAMA ...,GAANA_000761_2022_0028.pdf,2025-02-07 08:51:56.317,51,000761,2022,1,\n \n1 \n \n \nProcesso nº: 761/2...,pessoas=[PessoaAntecedentes(nome_pessoa='Maria...,"[{'nome': 'Maria de Fátima Borges Marinho', 'c...","[{'nome': 'Maria de Fátima Borges Marinho', 'c..."
1,GAANA,000913/2023,COMUNICAÇÃO DE IRREGULARIDADE ...,SIGILOSO ...,GAANA_000913_2023_0027.pdf,2025-02-17 12:20:47.240,46,000913,2023,1,\n \n1 \n \n \nProcesso nº: 913/2...,pessoas=[PessoaAntecedentes(nome_pessoa='Samue...,"[{'nome': 'Samuel Oliveira de Souto', 'cpf': '...","[{'nome': 'Samuel Oliveira de Souto', 'cpf': '..."
2,GAANA,001749/2019,"CONTAS DO CHEFE DO PODER EXECUTIVO MUNICIPAL, ...",PREFEITURA MUNICIPAL DE VIÇOSA ...,GAANA_001749_2019_0042.pdf,2025-02-17 12:29:42.660,80,001749,2019,1,\n \n1 \n \n \nProcesso nº: 1749/...,pessoas=[PessoaAntecedentes(nome_pessoa='Ana P...,"[{'nome': 'Ana Paula de Oliveira Gomes', 'cpf'...","[{'nome': 'Ana Paula de Oliveira Gomes', 'cpf'..."
3,GAANA,003277/2020,AUMENTO REMUNERATÓRIO DURANTE A PANDEMIA - REM...,CÂMARA MUNICIPAL DE ANTÔNIO MARTINS ...,GAANA_003277_2020_0124.pdf,2025-02-14 09:31:09.193,201,003277,2020,1,\n \n1 \n \n \nProcesso nº: 3277/...,pessoas=[PessoaAntecedentes(nome_pessoa='Jorge...,[{'nome': 'Jorge Vinicius de Oliveira Fernande...,[{'nome': 'Jorge Vinicius de Oliveira Fernande...
4,GAANA,003980/2022,"APURAÇÃO DE RESPONSABILIDADE, REFERENTE AO PRO...",PREFEITURA MUNICIPAL DE CEARÁ-MIRIM ...,GAANA_003980_2022_0029.pdf,2025-01-28 12:49:37.590,42,003980,2022,1,\n \n1 \n \n \nProcesso nº: 3980/...,pessoas=[PessoaAntecedentes(nome_pessoa='José ...,"[{'nome': 'José Walter Fernandes', 'cpf': '413...","[{'nome': 'José Walter Fernandes', 'cpf': '413..."
5,GAANA,004072/2021,MONITORAMENTO ...,AGÊNCIA DE FOMENTO DO RN S/A ...,GAANA_004072_2021_0039.pdf,2025-02-20 13:06:07.243,74,004072,2021,1,\n \n1 \n \n \nProcesso nº: 4072/...,pessoas=[PessoaAntecedentes(nome_pessoa='Ediva...,"[{'nome': 'Edivane Vilar de Souza', 'cpf': '45...","[{'nome': 'Edivane Vilar de Souza', 'cpf': '45..."
6,GAANA,004728/2022,DENÚNCIA ...,FRANCINALDO MOREIRA DA SILVA ...,GAANA_004728_2022_0049.pdf,2025-04-22 09:21:47.683,72,004728,2022,1,\n \n1 \n \n \nProcesso nº: 4728/...,pessoas=[PessoaAntecedentes(nome_pessoa='José ...,"[{'nome': 'José Alexandre Sobrinho', 'cpf': No...","[{'nome': 'José Alexandre Sobrinho', 'cpf': No..."
7,GAANA,005119/2021,APURAÇÃO DE RESPONSABILIDADE REFERENTE A ATRAS...,SECRETARIA MUNICIPAL DE MEIO AMBIENTE E URBANI...,GAANA_005119_2021_0016.pdf,2025-04-23 09:18:42.513,34,005119,2021,1,\n \n1 \n \n \nProcesso nº: 5119/...,pessoas=[PessoaAntecedentes(nome_pessoa='Thiag...,"[{'nome': 'Thiago de Paula Nunes Mesquita', 'c...","[{'nome': 'Thiago de Paula Nunes Mesquita', 'c..."
8,GAANA,005181/2020,AUDITORIA DE CONFORMIDADE-EXERCÍCIO 2018 ...,PREFEITURA MUNICIPAL DE MOSSORÓ ...,GAANA_005181_2020_0084.pdf,2025-02-18 08:39:55.740,110,005181,2020,1,\n \n1 \n \n \nProcesso nº: 5181/...,pessoas=[PessoaAntecedentes(nome_pessoa='Sebas...,"[{'nome': 'Sebastião Ronaldo Martins Cruz', 'c...","[{'nome': 'Sebastião Ronaldo Martins Cruz', 'c..."
9,GAANA,006621/2015,CONTAS DO CHEFE DO PODER EXECUTIVO REFERENTE A...,PREF.MUN.UPANEMA ...,GAANA_006621_2015_0037.pdf,2025-01-14 10:15:59.170,73,006621,2015,1,\n \n1 \n \n \nProcesso nº: 6621/...,pessoas=[PessoaAntecedentes(nome_pessoa='Luiz ...,"[{'nome': 'Luiz Jairo Bezerra de Mendonça', 'c...","[{'nome': 'Luiz Jairo Bezerra de Mendonça', 'c..."
